###  PyTorch implementation of the paper : Multi-view Integration Learning for Irregularly-sampled Clinical Time Series (MIAM) 

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_curve, roc_auc_score

import tensorflow as tf
import torch.nn.functional as F
from torch.autograd import Variable
import math
from attention_graph_util import *
import copy


from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve
from sklearn.model_selection import KFold



### Data Preprocessing and Transformation to MIAM Format

Using PhysioNet 2012 Challenge dataset, In-Hospital Mortality Prediction

In [138]:


# Define data folders and outcomes paths for each set
data_folder_a = "/media/usr/HDD/Data/EHR/Challenge_2012/set-a"
data_folder_b = "/media/usr/HDD/Data/EHR/Challenge_2012/set-b"
data_folder_c = "/media/usr/HDD/Data/EHR/Challenge_2012/set-c"
outcomes_path_a = "/media/usr/HDD/Data/EHR/Challenge_2012/Outcomes-a.txt"
outcomes_path_b = "/media/usr/HDD/Data/EHR/Challenge_2012/Outcomes-b.txt"
outcomes_path_c = "/media/usr/HDD/Data/EHR/Challenge_2012/Outcomes-c.txt"

def load_txt_data(folder_path):
    data_dict = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            record_id = file_name.split('.')[0]
            file_path = os.path.join(folder_path, file_name)
            df = pd.read_csv(file_path, sep=',')
            data_dict[record_id] = df
    return data_dict

def load_outcomes(outcomes_path):
    outcomes_df = pd.read_csv(outcomes_path, sep=',')
    outcomes_df.set_index('RecordID', inplace=True)
    return outcomes_df



from scipy.stats.mstats import winsorize

def apply_winsorize(data_dict):
    for record_id, df in data_dict.items():
        for column in df.columns:
            # 열이 비어 있지 않은지 확인
            if df[column].notna().sum() > 0:
                df[column] = winsorize(df[column], limits=[0.02, 0.02])
    return data_dict


def z_normalize(data_dict, mean_dict=None, std_dict=None, is_train=True):
    if is_train:
        mean_dict = {}
        std_dict = {}
        for record_id, df in data_dict.items():
            for column in df.columns:
                if column not in mean_dict:
                    mean_dict[column] = df[column].mean()
                    std_dict[column] = df[column].std()

    for record_id, df in data_dict.items():
        for column in df.columns:
            if std_dict[column] != 0:
                df[column] = (df[column] - mean_dict[column]) / std_dict[column]
    
    return data_dict, mean_dict, std_dict



# def preprocess_data_with_fixed_variables(data_dict, variable_list):
#     processed_data = {}
#     for record_id, df in data_dict.items():
#         df = df.groupby(['Time', 'Parameter'], as_index=False).mean()
#         df_pivot = df.pivot(index='Time', columns='Parameter', values='Value')
#         df_pivot = df_pivot.reindex(columns=variable_list, fill_value=np.nan)
#         processed_data[record_id] = df_pivot.reset_index(drop=True)
#     return processed_data

variable_list = [
    'DiasABP', 'NIDiasABP', 'FiO2', 'GCS',
'HR', 'MAP', 'NIMAP', 'PaCO2', 'PaO2',
'RespRate', 'SysABP', 'NISysABP', 'Temp', 'Urine',
'ALP', 'ALT', 'AST', 'Albumin',
'BUN', 'Bilirubin', 'Cholesterol', 'Creatinine', 'Glucose', 'HCO3', 'HCT',
'K', 'Lactate', 'Mg', 'Na', 'Platelets',
'SaO2', 'TropI', 'TropT', 'WBC', 'pH']


def preprocess_data_with_fixed_variables(data_dict, variable_list):
    processed_data = {}
    for record_id, df in data_dict.items():
        # 'RecordID', 'Age', 'Gender', 'Height', 'ICUType' 등 일반 정보 제외
        df = df[~df['Parameter'].isin(['RecordID', 'Age', 'Gender', 'Height', 'ICUType', 'Weight'])]
        
        # 'Time'과 'Parameter'로 groupby 후 평균을 계산해 중복 해결
        df = df.groupby(['Time', 'Parameter'], as_index=False).mean()

        # 피벗 테이블을 만들어 변수들을 열로 변환
        df_pivot = df.pivot(index='Time', columns='Parameter', values='Value')

        # 변수 리스트에 맞춰 열을 정렬하고, 누락된 변수는 NaN으로 채움
        df_pivot = df_pivot.reindex(columns=variable_list, fill_value=np.nan)

        # 처리된 데이터를 사전에 저장
        processed_data[record_id] = df_pivot.reset_index(drop=True)
    
    return processed_data



def map_outcomes(data_dict, outcomes_df):
    labeled_data = {}
    for record_id, df in data_dict.items():
        if int(record_id) in outcomes_df.index:
            outcome = outcomes_df.loc[int(record_id)]
            labeled_data[record_id] = (df, outcome)
    return labeled_data




In [155]:

# Assuming all previous functions and model definitions are included

# Load data for each set
data_a = load_txt_data(data_folder_a)
data_b = load_txt_data(data_folder_b)
data_c = load_txt_data(data_folder_c)

# Load outcomes for each set
outcomes_a = load_outcomes(outcomes_path_a)
outcomes_b = load_outcomes(outcomes_path_b)
outcomes_c = load_outcomes(outcomes_path_c)

data_a = preprocess_data_with_fixed_variables(data_a, variable_list)
data_b = preprocess_data_with_fixed_variables(data_b, variable_list)
data_c = preprocess_data_with_fixed_variables(data_c, variable_list)

# Preprocess the data
data_a = apply_winsorize(data_a)
data_a, mean_dict, std_dict = z_normalize(data_a, is_train=True)
data_b = apply_winsorize(data_b)
data_b, _, _ = z_normalize(data_b, mean_dict, std_dict, is_train=False)
data_c = apply_winsorize(data_c)
data_c, _, _ = z_normalize(data_c, mean_dict, std_dict, is_train=False)



# data_a = zero_imputation(data_a)
# data_b = zero_imputation(data_b)
# data_c = zero_imputation(data_c)

data_a_labeled = map_outcomes(data_a, outcomes_a)
data_b_labeled = map_outcomes(data_b, outcomes_b)
data_c_labeled = map_outcomes(data_c, outcomes_c)




In [156]:
print("Number of patients in filtered data_a:", len(data_a_labeled))
print("Number of patients in filtered data_b:", len(data_b_labeled))
print("Number of patients in filtered data_c:", len(data_c_labeled))

Number of patients in filtered data_a: 1416
Number of patients in filtered data_b: 1411
Number of patients in filtered data_c: 1424


In [157]:
# Function to find the maximum time-series length in a dataset
def get_max_time_series_length(data_set):
    max_length = max(patient_data[0].shape[0] for patient_data in data_set.values())  # patient_data[0] is the DataFrame
    return max_length

# Calculate and print the maximum time-series length for each dataset
max_length_a = get_max_time_series_length(data_a_labeled)
max_length_b = get_max_time_series_length(data_b_labeled)
max_length_c = get_max_time_series_length(data_c_labeled)

print("Maximum time-series length in data_a:", max_length_a)
print("Maximum time-series length in data_b:", max_length_b)
print("Maximum time-series length in data_c:", max_length_c)



Maximum time-series length in data_a: 202
Maximum time-series length in data_b: 185
Maximum time-series length in data_c: 214


In [218]:
# Ensure that outcomes_a.index is a string
outcomes_a.index = outcomes_a.index.astype(str)
outcomes_b.index = outcomes_b.index.astype(str)
outcomes_c.index = outcomes_c.index.astype(str)

# Filter outcomes based on the existing keys in filtered_data_a (which are already strings)
filtered_outcomes_a = outcomes_a.loc[outcomes_a.index.isin(filtered_data_a.keys())]
filtered_outcomes_b = outcomes_b.loc[outcomes_b.index.isin(filtered_data_b.keys())]
filtered_outcomes_c = outcomes_c.loc[outcomes_c.index.isin(filtered_data_c.keys())]

# Concatenate the filtered outcomes
filtered_outcomes = pd.concat([filtered_outcomes_a, filtered_outcomes_b, filtered_outcomes_c])

# Count positive and negative outcomes
positive_count = filtered_outcomes['In-hospital_death'].sum()
negative_count = len(filtered_outcomes) - positive_count

# Calculate the imbalance ratio
imbalance_ratio = negative_count / positive_count

# Display the results
positive_count, negative_count, imbalance_ratio


(616, 3635, 5.900974025974026)

In [220]:
def calculate_missing_rate(data_dict):
    total_values = 0
    missing_values = 0

    # Loop over each record (patient) in the dataset
    for record_id, (df, outcome) in data_dict.items():  # Access the first element in the tuple (df)
        total_values += df.size  # Total number of values (cells) in the DataFrame
        missing_values += df.isna().sum().sum()  # Count the NaN values as missing

    missing_rate = (missing_values / total_values) * 100
    return missing_rate

# Assuming data_a_labeled is the dataset after loading and preprocessing
missing_rate = calculate_missing_rate(data_a_labeled)
print(f"Missing Rate: {missing_rate:.2f}%")


Missing Rate: 85.61%


In [243]:
len(variable_list)

35

### Data Differences from paper 

- Maximum time series length: 214 (The paper says "The number of irregular time points ranged from 1 to 202")
- Positive_counts, negative_counts: 616, 3635 (in paper, 554 positive, 3,443 negative)
- Number of patients: 4251 (in paper, 3,997)
- Missing Rate : 85.61% (in paper, 80.5%)

## Model

In [256]:
class Multi_Duration_Pipeline_Residual(nn.Module):
    def __init__(self, input_dim, d_model, d_ff, num_stack, num_heads, max_length, n_iter):
        super().__init__()

        self.n_iter = n_iter

        # Embeddings
        self.obs_embed = Embedder(input_dim, d_model)
        self.mask_embed = Embedder(input_dim, d_model)
        self.deltas_embed = Embedder(input_dim, d_model)

        # Positional Encoding
        self.pe = PositionalEncoder_TimeDescriptor(d_model, max_length)

        # Encoding blocks
        self.obs_encoding_block = Encoding_Block(d_model, max_length, num_heads, d_ff, num_stack)
        self.mask_encoding_block = Encoding_Block(d_model, max_length, num_heads, d_ff, num_stack)
        self.deltas_encoding_block = Encoding_Block(d_model, max_length, num_heads, d_ff, num_stack)
        self.comb_encoding_block = Encoding_Block(d_model, max_length, num_heads, d_ff, num_stack)
        self.missing_comb_encoding_block = Encoding_Block(d_model, max_length, num_heads, d_ff, num_stack)

        # Decoder
        obs_embed_weight = self.obs_embed.embed.weight
        n, v = obs_embed_weight.size()
        self.decoder = nn.Linear(n, v, bias=False)
        self.decoder.weight.data = obs_embed_weight.transpose(1, 0)
        self.decoder_bias = nn.Parameter(torch.zeros(v))


        
        self.los_classifier = nn.Sequential(
            nn.Linear(d_model * 2, d_model),
            nn.BatchNorm1d(d_model),
            nn.LeakyReLU(),
            nn.Linear(d_model, 1),
        )

        self.classifier = nn.Sequential(
            nn.Linear(d_model * 2, d_model),
            nn.BatchNorm1d(d_model),
            nn.LeakyReLU(),
            nn.Linear(d_model, 1),
        )

        self.classifier2 = nn.Sequential(
            nn.Linear(input_dim * 2, input_dim),
            nn.BatchNorm1d(input_dim),
            # nn.LeakyReLU(),
            nn.Tanh(),
            nn.Linear(input_dim, 1))

        self.time_encoding_block = Encoding_Block(d_model, 3, num_heads, d_ff, num_stack)
        self.reset_parameters()

        self.lin_clsf = nn.Sequential(
            nn.Linear(d_model*2, 1),
        )

        self.dropout = nn.Dropout(0.3)


    def reset_parameters(self):
        for weight in self.parameters():
            if len(weight.size()) == 1:
                continue
            stv = 1. / math.sqrt(weight.size(1))
            nn.init.uniform_(weight, -stv, stv)

        
    def forward(self, data, mask, times, deltas, attn_mask):
        """
        :param src: Batch x Max_seq_len x Variable
        :param mask: Batch x Max_seq_len x Max_seq_len
        """

        # make attn_mask
        batch_size, seq_len, var_num = data.size()
        attn_mask = attn_mask.unsqueeze(1)
        attn_mask = attn_mask.expand(batch_size, seq_len, seq_len)

        # Datas
        d_z = data#[:, 0, :, :]

        # Input embedding
        x_z = self.obs_embed(d_z)
        m = self.mask_embed(mask)
        d = self.deltas_embed(deltas)

        # Positional encoding
        x_z, m, d = self.pe(x_z, m, d, times)

        # obs_mha, mask_mha, delta_mha
        x_z = self.obs_encoding_block(x_z, x_z, attn_mask)
        x_s = x_z
        m = self.mask_encoding_block(m, m, attn_mask)
        d = self.deltas_encoding_block(d, d, attn_mask)
        missing_comb_z = self.missing_comb_encoding_block(d, m, attn_mask)


        # Attention Distillation
        for n in range(self.n_iter):
            comb_z = self.comb_encoding_block(missing_comb_z, x_z, attn_mask)
            x_z = self.obs_encoding_block(comb_z, x_z, attn_mask)


        """Imputation Part"""
        # Input Embedding
        x_mskd = self.obs_embed(d_z.to(data.device))
        m_mskd = self.mask_embed(mask.to(data.device))
        d_mskd = self.deltas_embed(deltas)

        # Positional encoding
        x_mskd, m_mskd, d_mskd = self.pe(x_mskd, m_mskd, d_mskd, times)

        # Masked MHA
        x_d = self.obs_encoding_block(x_mskd, x_mskd, attn_mask)

        # Encoder-decoder Attention
        x_d = self.obs_encoding_block(x_z, x_d, attn_mask)
        x_final = x_d + x_z

        x_dd = self.decoder(x_final) + self.decoder_bias

        # Classification
        combine = 1
        x_avg = x_z.mean(dim=1)
        m_avg = missing_comb_z.mean(dim=1)#m_final.mean(dim=1)
        x_m_cat = torch.stack((x_avg, m_avg), 1).reshape([x_avg.shape[0], -1])
        out = self.classifier(x_m_cat)
        reg_out = self.los_classifier(x_m_cat)
        y = torch.sigmoid(out).squeeze(-1)
        return reg_out.squeeze(-1), y, x_dd# reg_out out.squeeze(-1) # y


class Encoding_Block(nn.Module):
    def __init__(self, d_model, max_length, num_heads, d_ff, num_stack):
        super().__init__()

        self.N = num_stack

        self.layers = get_clones(EncoderLayer(d_model, max_length, num_heads, d_ff), num_stack)
        self.norm = Norm(d_model)

    def forward(self, q, k, attn_mask):
        # MHA Encoding
        for i in range(self.N):
            q, k = self.layers[i](q, k, attn_mask)

        # Normalize
        encoded_data = self.norm(q)
        return encoded_data

class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        # self.embed = nn.Embedding(vocab_size, d_model)
        self.embed = nn.Linear(vocab_size, d_model)
        # self.embed = nn.Sequential(
        #     nn.Linear(vocab_size, 32),
        #     # nn.BatchNorm1d(128),
        #     nn.LeakyReLU(),
        #     nn.Linear(32, d_model),
        # )

    def forward(self, x):
        return self.embed(x)

class PositionalEncoder_TimeDescriptor(nn.Module):
    def __init__(self, d_model, max_seq_len):
        super().__init__()
        self.d_model = d_model
        self.max_seq_len = max_seq_len

    def get_sinusoid_encoding_table(self, seq_len, d_model):
        def cal_angle(position, hid_idx):
            return position / np.power(self.max_seq_len, 2 * (hid_idx // 2) / d_model)

        def get_posi_angle_vec(position):
            return [cal_angle(position, hid_j) for hid_j in range(d_model)]

        sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(seq_len)])

        # Apply sin to even indices, cos to odd indices
        sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # dim 2i
        sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # dim 2i+1
        return torch.FloatTensor(sinusoid_table)

    def time_encoding(self, t, seq_len):
        batch_size = t.size(0)
        pe = self.get_sinusoid_encoding_table(seq_len, self.d_model)
        pe = pe.unsqueeze(0).expand(batch_size, -1, -1)  # Expand to shape [batch_size, seq_len, d_model]
        return pe

    def forward(self, x, m, delta, t):
        x = x * math.sqrt(self.d_model)
        m = m * math.sqrt(self.d_model)
        delta = delta * math.sqrt(self.d_model)

        seq_len = x.size(1)  # Get sequence length
        pos = self.time_encoding(t, seq_len)  # Pass t and seq_len

        # Ensure pos is moved to the same device as x
        pos = pos.to(x.device)  # Move pos to the same device as x (e.g., GPU if x is on GPU)

        # Apply position encoding to x, m, delta
        x = x + Variable(pos, requires_grad=False)
        m = m + Variable(pos, requires_grad=False)
        delta = delta + Variable(pos, requires_grad=False)

        return x, m, delta

    

def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class Norm(nn.Module):
    def __init__(self, d_model, eps=1e-6):
        super().__init__()
        self.size = d_model

        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps

    def forward(self, x):
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) \
               / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm


class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout=0.2):
        super().__init__()

        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads

        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, tp, mask=None):
        bs = q.size(0)  # Batch size
        seq_len = q.size(2)  # Sequence length

        # Linear transformations and split into h heads
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)

        # Transpose to get dimensions bs * h * seq_len * d_k
        k = k.transpose(1, 2)  # [bs, h, seq_len, d_k]
        q = q.transpose(1, 2)  # [bs, h, seq_len, d_k]
        v = v.transpose(1, 2)  # [bs, h, seq_len, d_k]

        # Adjust the mask to match sequence length
        if mask is not None:
            if mask.dim() == 2:  # If mask is [bs, seq_len], unsqueeze and repeat
                mask = mask.unsqueeze(1).unsqueeze(2)  # Convert to [bs, 1, 1, seq_len]
            elif mask.dim() == 3:  # If mask is [bs, seq_len, seq_len]
                mask = mask.unsqueeze(1)  # Convert to [bs, 1, seq_len, seq_len]

            # Now slice the mask to match the sequence length
            seq_len = q.size(2)
            mask = mask[:, :, :seq_len, :seq_len]  # Slice to fit the seq_len of q, k, and v

            # Expand mask to match the batch size (bs) and number of heads (self.h)
            mask = mask.expand(bs, self.h, seq_len, seq_len)

        # Calculate attention scores using the adjusted mask
        scores = attention(q, k, v, self.d_k, mask, self.dropout)
        if tp == 1:
            scores = scores.transpose(2, 3)

        # Concatenate heads and put through final linear layer
        concat = scores.transpose(1, 2).contiguous().view(bs, -1, self.d_model)
        output = self.out(concat)

        return output

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        # We set d_ff as a default to 2048
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

def attention(q, k, v, d_k, mask=None, dropout=None):
    """
    :param q: Batch x n_head x max_seq_len x variable
    :param k: Batch x n_head x max_seq_len x variable
    :param v: Batch x n_head x max_seq_len x variable
    :param d_k:
    :param mask: Batch x n_had x max_seq_len x max_seq_len
    :param dropout:
    :return:
    """

    scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # [Batch x n_head x max_seq_len x max_seq_len]
    #print(f'scores shape: {scores.shape}, mask shape: {mask.shape}')
    if mask is not None:
        scores = scores.masked_fill(mask, -1e9)
    scores = F.softmax(scores, dim=-1)

    if dropout is not None:
        scores = dropout(scores)

    output = torch.matmul(scores, v)
    return output



class EncoderLayer(nn.Module):
    def __init__(self, d_model, max_length, heads, d_ff, dropout=0):
        super().__init__()
        self.norm_q = Norm(d_model)
        self.norm_k = Norm(d_model)
        self.norm_q_attn = Norm(d_model)

        self.attn = MultiHeadAttention(heads, d_model)
        self.time_attn = MultiHeadAttention(heads, d_model)
        self.var_attn = MultiHeadAttention(heads, d_model)

        self.ff = FeedForward(d_model, d_ff)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)

        self.cat_lin = nn.Linear(128, 64)

    def forward(self, q, k, mask):
        q2 = self.norm_q(q)
        k2 = self.norm_k(k)
        q = q + self.dropout_1(self.attn(q2, k2, k2, 0, mask))
        q2 = self.norm_q_attn(q)
        q = q + self.dropout_2(self.ff(q2))

        return q, k

In [250]:


class Multi_Duration_Pipeline_Residual(nn.Module):
    def __init__(self, input_dim, d_model, d_ff, num_stack, num_heads, max_length, n_iter):
        super().__init__()

        self.n_iter = n_iter

        # Embeddings
        self.obs_embed = Embedder(input_dim, d_model)
        self.mask_embed = Embedder(input_dim, d_model)
        self.deltas_embed = Embedder(input_dim, d_model)

        # Positional Encoding
        self.pe = PositionalEncoder_TimeDescriptor(d_model, max_length)

        # Encoding blocks
        self.obs_encoding_block = Encoding_Block(d_model, max_length, num_heads, d_ff, num_stack)
        self.mask_encoding_block = Encoding_Block(d_model, max_length, num_heads, d_ff, num_stack)
        self.deltas_encoding_block = Encoding_Block(d_model, max_length, num_heads, d_ff, num_stack)
        self.comb_encoding_block = Encoding_Block(d_model, max_length, num_heads, d_ff, num_stack)
        self.missing_comb_encoding_block = Encoding_Block(d_model, max_length, num_heads, d_ff, num_stack)

        # Decoder
        obs_embed_weight = self.obs_embed.embed.weight
        n, v = obs_embed_weight.size()
        self.decoder = nn.Linear(n, v, bias=False)
        self.decoder.weight.data = obs_embed_weight.transpose(1, 0)
        self.decoder_bias = nn.Parameter(torch.zeros(v))

        # Classification layers
        self.classifier = nn.Sequential(
            nn.Linear(d_model * 2, d_model),
            nn.BatchNorm1d(d_model),
            nn.Tanh(),
            nn.Linear(d_model, 1),
        )

        # Initialize parameters
        self.reset_parameters()

    def reset_parameters(self):
        for weight in self.parameters():
            if len(weight.size()) == 1:
                continue
            stv = 1. / math.sqrt(weight.size(1))
            nn.init.uniform_(weight, -stv, stv)

        
    def forward(self, data, mask, times, deltas, attn_mask):
        batch_size, seq_len, var_num = data.size()

        # Input embeddings
        x_z = self.obs_embed(data)
        m = self.mask_embed(mask)
        d = self.deltas_embed(deltas)

        # Positional encoding
        x_z, m, d = self.pe(x_z, m, d, times)  # Remove seq_len since it's not needed here

        # Proceed with encoding blocks
        x_z = self.obs_encoding_block(x_z, x_z, attn_mask)
        m = self.mask_encoding_block(m, m, attn_mask)
        d = self.deltas_encoding_block(d, d, attn_mask)
        missing_comb_z = self.missing_comb_encoding_block(d, m, attn_mask)

        # Attention distillation
        for n in range(self.n_iter):
            comb_z = self.comb_encoding_block(missing_comb_z, x_z, attn_mask)
            x_z = self.obs_encoding_block(comb_z, x_z, attn_mask)
            missing_comb_z = self.missing_comb_encoding_block(missing_comb_z, missing_comb_z, attn_mask)

        # Imputation
        x_final = x_z + missing_comb_z
        x_dd = self.decoder(x_final) + self.decoder_bias  # Imputed output

        # Classification
        x_avg = x_z.mean(dim=1)
        m_avg = missing_comb_z.mean(dim=1)
        x_m_cat = torch.cat((x_avg, m_avg), dim=1)
        out = self.classifier(x_m_cat)
        y = torch.sigmoid(out).squeeze(-1)

        return y, x_dd  # Returning both classifier output and imputed data


class Encoding_Block(nn.Module):
    def __init__(self, d_model, max_length, num_heads, d_ff, num_stack):
        super().__init__()

        self.N = num_stack

        self.layers = get_clones(EncoderLayer(d_model, max_length, num_heads, d_ff), num_stack)
        self.norm = Norm(d_model)

    def forward(self, q, k, attn_mask):
        # MHA Encoding
        for i in range(self.N):
            q, k = self.layers[i](q, k, attn_mask)

        # Normalize
        encoded_data = self.norm(q)
        return encoded_data

class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        # self.embed = nn.Embedding(vocab_size, d_model)
        self.embed = nn.Linear(vocab_size, d_model)
        # self.embed = nn.Sequential(
        #     nn.Linear(vocab_size, 32),
        #     # nn.BatchNorm1d(128),
        #     nn.LeakyReLU(),
        #     nn.Linear(32, d_model),
        # )

    def forward(self, x):
        return self.embed(x)

class PositionalEncoder_TimeDescriptor(nn.Module):
    def __init__(self, d_model, max_seq_len):
        super().__init__()
        self.d_model = d_model
        self.max_seq_len = max_seq_len

    def get_sinusoid_encoding_table(self, seq_len, d_model):
        def cal_angle(position, hid_idx):
            return position / np.power(self.max_seq_len, 2 * (hid_idx // 2) / d_model)

        def get_posi_angle_vec(position):
            return [cal_angle(position, hid_j) for hid_j in range(d_model)]

        sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(seq_len)])

        # Apply sin to even indices, cos to odd indices
        sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # dim 2i
        sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # dim 2i+1
        return torch.FloatTensor(sinusoid_table)

    def time_encoding(self, t, seq_len):
        batch_size = t.size(0)
        pe = self.get_sinusoid_encoding_table(seq_len, self.d_model)
        pe = pe.unsqueeze(0).expand(batch_size, -1, -1)  # Expand to shape [batch_size, seq_len, d_model]
        return pe

    def forward(self, x, m, delta, t):
        x = x * math.sqrt(self.d_model)
        m = m * math.sqrt(self.d_model)
        delta = delta * math.sqrt(self.d_model)

        seq_len = x.size(1)  # Get sequence length
        pos = self.time_encoding(t, seq_len)  # Pass t and seq_len

        # Ensure pos is moved to the same device as x
        pos = pos.to(x.device)  # Move pos to the same device as x (e.g., GPU if x is on GPU)

        # Apply position encoding to x, m, delta
        x = x + Variable(pos, requires_grad=False)
        m = m + Variable(pos, requires_grad=False)
        delta = delta + Variable(pos, requires_grad=False)

        return x, m, delta

    

def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class Norm(nn.Module):
    def __init__(self, d_model, eps=1e-6):
        super().__init__()
        self.size = d_model

        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps

    def forward(self, x):
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) \
               / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm


class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout=0.2):
        super().__init__()

        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads

        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, tp, mask=None):
        bs = q.size(0)

        # perform linear operation and split into h heads
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)  # [batch_size * len_q * n_heads * hidden_dim]
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)  # [batch_size * len_q * n_heads * hidden_dim]
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)  # [batch_size * len_q * n_heads * hidden_dim]

        # transpose to get dimensions bs * h * sl * d_model
        k = k.transpose(1, 2)  # [batch_size * n_heads * len_q * hidden_dim]
        q = q.transpose(1, 2)  # [batch_size * n_heads * len_q * hidden_dim]
        v = v.transpose(1, 2)  # [batch_size * n_heads * len_q * hidden_dim]

        if mask is not None:
            mask = mask.unsqueeze(1).repeat(1, self.h, 1, 1)  # [batch_size x n_heads x len_q x len_k]


        if tp == 1:  # if transpose
            k = k.transpose(2, 3)  # [batch_size * n_heads * hidden_dim * len_q]
            q = q.transpose(2, 3)  # [batch_size * n_heads * hidden_dim * len_q]
            v = v.transpose(2, 3)  # [batch_size * n_heads * hidden_dim * len_q]


        # calculate attention using function we will define next
        scores = attention(q, k, v, self.d_k, mask, self.dropout)
        if tf == 1:
            scores = scores.transpose(2, 3)

        # concatenate heads and put through final linear layer
        concat = scores.transpose(1, 2).contiguous() \
            .view(bs, -1, self.d_model)

        output = self.out(concat)

        return output

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        # We set d_ff as a default to 2048
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

def attention(q, k, v, d_k, mask=None, dropout=None):
    """
    :param q: Batch x n_head x max_seq_len x variable
    :param k: Batch x n_head x max_seq_len x variable
    :param v: Batch x n_head x max_seq_len x variable
    :param d_k:
    :param mask: Batch x n_had x max_seq_len x max_seq_len
    :param dropout:
    :return:
    """

    scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # [Batch x n_head x max_seq_len x max_seq_len]

    # Attention calculation 전에 attn_mask를 Boolean으로 변환합니다
    if mask is not None:
        mask = mask.bool()  # Ensure mask is boolean
        scores = scores.masked_fill(mask, -1e9)
    scores = F.softmax(scores, dim=-1)

    if dropout is not None:
        scores = dropout(scores)

    output = torch.matmul(scores, v)
    return output


class EncoderLayer(nn.Module):
    def __init__(self, d_model, max_length, heads, d_ff, dropout=0):
        super().__init__()
        self.norm_q = Norm(d_model)
        self.norm_k = Norm(d_model)
        self.norm_q_attn = Norm(d_model)

        self.attn = MultiHeadAttention(heads, d_model)
        self.time_attn = MultiHeadAttention(heads, d_model)
        self.var_attn = MultiHeadAttention(heads, d_model)

        self.ff = FeedForward(d_model, d_ff)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)

        self.cat_lin = nn.Linear(128, 64)

    def forward(self, q, k, mask):
        q2 = self.norm_q(q)
        k2 = self.norm_k(k)
        q = q + self.dropout_1(self.attn(q2, k2, k2, 0, mask))
        q2 = self.norm_q_attn(q)
        q = q + self.dropout_2(self.ff(q2))

        return q, k

## Loss function

In [18]:
class FocalLoss(nn.Module):
    def __init__(self,  lambda1, device, alpha=1, gamma=0, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce
        self.device = device
        self.lambda1 = torch.tensor(lambda1).to(device)

    def forward(self, model, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-1*BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        # Regularization
        l1_regularization = torch.tensor(0).float().to(self.device)
        for param in model.parameters():
            l1_regularization += torch.norm(param.to(self.device), 1)

        # Take the average
        loss = torch.mean(F_loss) + (self.lambda1 * l1_regularization)

        return loss


class WeightedBCE(nn.Module):
    def __init__(self,  device):
        super(WeightedBCE, self).__init__()
        self.device = device

    def forward(self, model, inputs, targets):
        inputs = inputs.detach().cpu()
        targets = targets.detach().cpu()

        pos_num = len(np.where(targets == 1)[0])
        neg_num = len(np.where(targets == 0)[0])
        if pos_num == 0:
            pos_weight = 1.0
        else:
            pos_weight = neg_num / pos_num
        weights = torch.zeros(len(targets))

        for i in range(len(targets)):
            if i == 1:
                weights[i] = pos_weight
            else:
                weights[i] = 1.0

        loss = F.binary_cross_entropy_with_logits(inputs, targets, pos_weight=weights)

        return loss.to(self.device)

## Optimizer

In [19]:
from torch.optim.optimizer import Optimizer

class RAdam(Optimizer):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.buffer = [[None, None, None] for ind in range(10)]
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = self.buffer[int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = math.sqrt(
                            (1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (
                                        N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        step_size = 1.0 / (1 - beta1 ** state['step'])
                    buffered[2] = step_size

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                if N_sma >= 5:
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size * group['lr'], exp_avg, denom)
                else:
                    p_data_fp32.add_(-step_size * group['lr'], exp_avg)

                p.data.copy_(p_data_fp32)

        return loss

In [273]:
import torch
torch.cuda.empty_cache()


In [267]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [268]:
device

device(type='cuda')

## Train & Validate

In [5]:
import random
# Seed
manualSeed = 64
np.random.seed(manualSeed)
torch.manual_seed(manualSeed)
random.seed(manualSeed)
torch.cuda.manual_seed(manualSeed)
torch.cuda.manual_seed_all(manualSeed)

In [297]:
def collate_fn(recs):
    rec_dict = {'values': torch.FloatTensor(np.array([r['values'] for r in recs])),
                'masks': torch.FloatTensor(np.array([r['masks'] for r in recs])),
                'deltas': torch.FloatTensor(np.array([r['deltas'] for r in recs])),
                'times': torch.FloatTensor(np.array([r['times'] for r in recs])),
                'labels': torch.FloatTensor(np.array([r['labels'] for r in recs]))
                }
    return rec_dict


from torch.nn.utils.rnn import pad_sequence

def collate_fns(recs):
    # Pad values, masks, deltas, and times using pad_sequence
    values = [r['values'] for r in recs]
    masks = [r['masks'] for r in recs]
    deltas = [r['deltas'] for r in recs]
    times = [r['times'] for r in recs]
    
    # Apply padding for variable length sequences
    values_padded = pad_sequence(values, batch_first=True)
    masks_padded = pad_sequence(masks, batch_first=True)
    deltas_padded = pad_sequence(deltas, batch_first=True)
    times_padded = pad_sequence(times, batch_first=True)
    
    # Convert the labels (In-hospital death and LOS) to tensors directly (no padding needed)
    labels = torch.FloatTensor([r['labels'] for r in recs])
    #labels = torch.FloatTensor([r['labels'] for r in recs]).squeeze(-1)
    los_labels = torch.FloatTensor([r['los_labels'] for r in recs])#.squeeze(-1)
    
    rec_dict = {
        'values': values_padded,
        'masks': masks_padded,
        'deltas': deltas_padded,
        'times': times_padded,
        'labels': labels,
        'los_labels': los_labels
    }
    return rec_dict


In [298]:
def create_dataloader(data_labeled, batch_size):
    data_list = []

    for _, (df, outcome) in data_labeled.items():
        values = torch.tensor(df.values, dtype=torch.float32)
        
        # Generate mask for observed (1) and missing (0) values
        mask = ~torch.isnan(values)
        mask = mask.float()

        # Replace NaNs in values with zero for processing
        values = torch.nan_to_num(values, nan=0.0)

        # Calculate absolute time steps and deltas
        times = torch.arange(values.size(0)).unsqueeze(-1).repeat(1, values.size(1))
        deltas = torch.zeros_like(times, dtype=torch.float32)
        for t in range(1, times.size(0)):
            for d in range(values.size(1)):
                if mask[t-1, d] == 0:
                    deltas[t, d] = deltas[t-1, d] + (times[t, d] - times[t-1, d])
                else:
                    deltas[t, d] = times[t, d] - times[t-1, d]

        label = outcome[['In-hospital_death']].values[0]
        los_label = outcome[['Length_of_stay']].values[0]

        # Append all values to the list
        data_list.append({'values': values, 'masks': mask, 'times': times, 'deltas': deltas, 'labels': label, 'los_labels': los_label})

    # Create the DataLoader with the updated collate function
    dataset = data_list
    return DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=True, collate_fn=collate_fns)




In [291]:
# Hyperparameters and initialization
input_dim = 35  # Number of time-series variables
d_model = 64  # Embedding dimension
d_ff = 128  # Feedforward dimension
num_stack = 2 # Number of encoder layers
num_heads = 8  # Number of attention heads
max_length = 215  # Maximum sequence length
n_iter = 3  # Number of distillation iterations
num_epochs = 60

In [292]:
alpha = 9
# gamma = 0.1
# # Loss rates
# beta = 0.1
# delta = 11

gamma = 0.15
# Loss rates
lambda_1 = 0
lambda_2 = 1
lambda_3 = 1

In [293]:
lambda1 = 5e-4
learning_rate = 5e-4
lr_decay = 10
lr_ratio = 0.2
batch_size = 64

In [294]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=1):
        self.patience = patience
        self.verbose = verbose
        self.best_auc = None
        self.counter = 0

    def step(self, current_auc):
        if self.best_auc is None:
            self.best_auc = current_auc
            return False  # Not yet stopped
        elif current_auc > self.best_auc:
            self.best_auc = current_auc
            self.counter = 0  # Reset counter
            return False  # Not stopped
        else:
            self.counter += 1
            if self.counter >= self.patience:
                if self.verbose:
                    print("Early stopping activated!")
                return True  # Stop training
            return False  # Not stopped

In [290]:
def train_and_validate(train_loader, valid_loader, input_dim, d_model, d_ff, num_stack, num_heads, max_length, n_iter, num_epochs, device, lambda_1, lambda_2, lambda_3, gamma, alpha, lr_decay, lr_ratio, batch_size):
    # Initialize model and optimizer
    model = Multi_Duration_Pipeline_Residual(input_dim, d_model, d_ff, num_stack, num_heads, max_length, n_iter).to(device)

    optimizer = torch.optim.RAdam(model.parameters(), lr=0.005)
    criterion_focal = FocalLoss(lambda1=1.0, device=device, gamma=gamma, alpha=alpha, logits=False).to(device)
    criterion_mse = nn.MSELoss()
    scheduler = StepLR(optimizer, step_size=lr_decay, gamma=lr_ratio)
    
    best_valid_auc = 0  # Track best validation AUC
    early_stopping = EarlyStopping(patience=20)  # Early stopping initialization
    

    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        train_loss = 0
        n_batches = 0
        
        # Training loop
        for batch_idx, data in enumerate(train_loader):
            x = data['values'].to(device)  # Batch x Time x Variable
            m = data['masks'].to(device)  # Batch x Time x Variable
            deltas = data['deltas'].to(device)  # Batch x Time x Variable
            times = data['times'].to(device)  # Batch x Time x Variable
            y = data['labels'].to(device)  # In-hospital death
            y1 = data['los_labels'].to(device)  # Length of stay
            
            # Zero Grad
            optimizer.zero_grad()
            
            # Create attention mask (assuming mask is based on deltas)
            attn_mask = deltas.data.eq(0)[:, :, 0]
            attn_mask[:, 0] = 0
            
            # Forward pass through the model
            los_out, y_hat, out = model(x, m, times, deltas, attn_mask)
            
            # Ensure correct shapes for loss calculation
            y1_target = y1 if y1.dim() == 2 else y1.unsqueeze(-1)  # Ensure y1 is 2D
            y_target = y if y.dim() == 2 else y.unsqueeze(-1)  # Ensure y is 2D
            
            # Loss calculation
            # Ensure y_hat and y_target have the same shape
            if y_hat.shape != y_target.shape:
                y_target = y_target.view_as(y_hat)

            
            loss_los = criterion_mse(los_out, y1_target[:, -1])  # Length of stay loss
            loss_cls = criterion_focal(model, y_hat, y_target[:, -1])  # In-hospital death loss
            loss_imp = criterion_mse(out, x)  # Imputation loss
            
            # Total loss
            loss = lambda_1 * loss_cls + lambda_2 * loss_los + lambda_3 * loss_imp
            train_loss += loss.item()
            
            # Backpropagation
            loss.backward()
            optimizer.step()
            n_batches += 1

        train_loss /= n_batches
        print(f"Epoch [{epoch + 1}/{num_epochs}], Training Loss: {train_loss:.4f}")

        # Validation process
        model.eval()  # Set model to evaluation mode
        valid_loss = 0
        valid_labels = []
        valid_predictions = []
        valid_los_predictions = []
        valid_los_labels = []

        with torch.no_grad():
            for batch in valid_loader:
                x = batch['values'].to(device)
                m = batch['masks'].to(device)
                deltas = batch['deltas'].to(device)
                times = batch['times'].to(device)
                y = batch['labels'].to(device)
                y1 = batch['los_labels'].to(device)
                
                attn_mask = deltas.data.eq(0)[:, :, 0]
                attn_mask[:, 0] = 0
                
                los_out, y_hat, out = model(x, m, times, deltas, attn_mask)

                # Ensure correct shapes for loss calculation
                y1_target = y1 if y1.dim() == 2 else y1.unsqueeze(-1)  # Ensure y1 is 2D
                y_target = y if y.dim() == 2 else y.unsqueeze(-1)  # Ensure y is 2D

                # Loss calculation
                loss_los = criterion_mse(los_out, y1_target[:, -1])  # Length of stay loss
                if y_hat.shape != y_target.shape:
                    y_target = y_target.view_as(y_hat)
                loss_cls = criterion_focal(model, y_hat, y_target[:, -1])  # In-hospital death loss
                loss_imp = criterion_mse(out, x)  # Imputation loss

                valid_loss += (lambda_1 * loss_cls + lambda_2 * loss_los + lambda_3 * loss_imp).item()

                # Collect labels and predictions
                valid_labels.append(y.cpu().numpy())
                valid_predictions.append(y_hat.cpu().numpy())
                valid_los_labels.append(y1.cpu().numpy())
                valid_los_predictions.append(los_out.cpu().numpy())
                
        valid_loss /= len(valid_loader)
        print(f"Epoch [{epoch + 1}/{num_epochs}], Validation Loss: {valid_loss:.4f}")
        
        # Early stopping logic
        # Add early stopping based on validation AUC or loss as per your implementation
        
        scheduler.step()  # Update learning rate
    
    print("Training and validation complete.")


# Convert to DataLoader
train_loader = create_dataloader(data_a_labeled, batch_size)
valid_loader = create_dataloader(data_b_labeled, batch_size)
    
# Assuming data loaders are initialized
train_and_validate(train_loader, valid_loader, input_dim, d_model, d_ff, num_stack, num_heads, max_length, n_iter, num_epochs, device, lambda_1, lambda_2, lambda_3, gamma, alpha, lr_decay, lr_ratio, batch_size)


___

The model part is taken from the authors' code, but the performance is different from the performance described in the paper.

There are several reasons for the discrepancy, such as the absence of benchmark parameters in the author's published code and the fact that the actual downloaded data differs from the description in the paper.

In [271]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True